In [13]:
%load_ext autoreload
%autoreload 2

###Importing all the useful librairies

import time
import json
import bz2
import pandas as pd
import numpy as np
import re
from pathlib import Path
import matplotlib.pyplot as plt
import pyarrow as pa
import pyarrow.parquet as pq
import glob

import warnings; warnings.simplefilter('ignore')
import os, codecs, string, random
import numpy as np
from numpy.random import seed as random_seed
from numpy.random import shuffle as random_shuffle
import matplotlib.pyplot as plt
%matplotlib inline  

seed = 42
random.seed(seed)
np.random.seed(seed)

#NLP libraries
import spacy, nltk, gensim, sklearn
#import pyLDAvis.gensim

#Vader
import vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

#Scikit imports
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
df = pd.read_csv("tot_2020.csv")

In [26]:
df.head(10)

,0,Q5268447,2020-01-17-000357,male,[ The delay ] will have an impact [ on Slough ] but that might be mitigated by the fact we are going to have this Western Rail Link to Heathrow. It looks like that may come in sooner than Crossrail.
0,1,Q4864119,2020-04-02-000239,male,[ The scheme ] treats addiction as an illness ...
1,2,Q816459,2020-03-19-000276,male,[ These ] actions will allow households who ha...
2,3,Q56255401,2020-01-08-000594,male,"11 straight weeks of pre-season,"
3,4,Q26923564,2020-02-21-000455,male,2019 was a landmark year for Fiverr as we comp...
4,5,Q4749380,2020-04-01-000532,female,7pm is when most hospitals change shifts. That...
5,6,Q5106592,2020-01-29-001076,male,"A collaborative opioid stewardship program, ac..."
6,7,Q771586,2020-01-26-000499,male,a few of the candidates who will do better in ...
7,8,Q16195572,2020-02-29-000491,male,a lack of resources and government funding
8,9,Q7322163,2020-03-23-000896,female,A little over a week old. Thankful for God's g...
9,10,Q18151420,2020-01-10-001448,male,a lot of contacts and a lot of knowledge


In [28]:
df.columns = ['id','quids', 'quoteID', 'gender', 'quote']

In [29]:
df

,id,quids,quoteID,gender,quote
0,1,Q4864119,2020-04-02-000239,male,[ The scheme ] treats addiction as an illness ...
1,2,Q816459,2020-03-19-000276,male,[ These ] actions will allow households who ha...
2,3,Q56255401,2020-01-08-000594,male,"11 straight weeks of pre-season,"
3,4,Q26923564,2020-02-21-000455,male,2019 was a landmark year for Fiverr as we comp...
4,5,Q4749380,2020-04-01-000532,female,7pm is when most hospitals change shifts. That...
...,...,...,...,...,...
701508,701509,Q33190271,2020-03-24-093541,male,You can't get social distance in a submarine a...
701509,701510,Q15897,2020-03-28-040445,female,You Know I' m No Good
701510,701511,Q495980,2020-02-02-058890,male,You understood the structure of our story and ...
701511,701512,Q3856214,2020-01-28-115305,male,You'd probably be able to do that every now an...


In [33]:
nlp = spacy.load('en_core_web_sm')

In [60]:
all_quotes = df.drop(['id','quids','quoteID','gender'], axis = 1)

In [61]:
all_quotes.head()

,quote
0,[ The scheme ] treats addiction as an illness ...
1,[ These ] actions will allow households who ha...
2,"11 straight weeks of pre-season,"
3,2019 was a landmark year for Fiverr as we comp...
4,7pm is when most hospitals change shifts. That...


In [45]:
df.to_csv(r'c:\data\pandas.txt', header=None, index=None, sep=' ', mode='a')

In [39]:
#type(df["quote"])

#np.savetxt(r'c:\data\np.txt', df.to_string())

pandas.core.series.Series

In [59]:
books = list()
corpus_root = "c:/data/"


with codecs.open(os.path.join(corpus_root,'pandas.txt'),encoding="utf8") as f:
    books.append(f.read())

In [55]:
print(books[0][0:600])

1 Q4864119 2020-04-02-000239 male "[ The scheme ] treats addiction as an illness and the results so far have been extremely encouraging."
2 Q816459 2020-03-19-000276 male "[ These ] actions will allow households who have an FHA-insured mortgage to meet the challenges of COVID-19 without fear of losing their homes, and help steady market concerns,"
3 Q56255401 2020-01-08-000594 male "11 straight weeks of pre-season,"
4 Q26923564 2020-02-21-000455 male "2019 was a landmark year for Fiverr as we completed a successful IPO, expanded the Fiverr ecosystem with new products, increased our internat
